In [2]:
import numpy as np
import pandas as pd
pd.plotting.register_matplotlib_converters()
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [3]:
df = pd.read_csv('Anime.csv')

In [4]:
df.head()

,ID,Title,Synonyms,Japanese,English,Synopsis,Type,Episodes,Status,Start_Aired,...,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
0,16498,Shingeki no Kyojin,"AoT, SnK",進撃の巨人,Attack on Titan,"Centuries ago, mankind was slaughtered to near...",TV,25.0,Finished Airing,"Apr 7, 2013",...,"Gore, Military, Survival",Shounen,24.0,R - 17+ (violence & profanity),8.531,519803.0,1002.0,1,3524109,155695
1,1535,Death Note,DN,デスノート,Death Note,"Brutal murders, petty thefts, and senseless vi...",TV,37.0,Finished Airing,"Oct 4, 2006",...,Psychological,Shounen,23.0,R - 17+ (violence & profanity),8.621,485487.0,732.0,2,3504535,159701
2,5114,Fullmetal Alchemist: Brotherhood,"Hagane no Renkinjutsushi Fullmetal Alchemist, ...",鋼の錬金術師 FULLMETAL ALCHEMIST,Fullmetal Alchemist Brotherhood,After a horrific alchemy experiment goes wrong...,TV,64.0,Finished Airing,"Apr 5, 2009",...,Military,Shounen,24.0,R - 17+ (violence & profanity),9.131,900398.0,12.0,3,2978455,207772
3,30276,One Punch Man,"One Punch-Man, One-Punch Man, OPM",ワンパンマン,One Punch Man,The seemingly unimpressive Saitama has a rathe...,TV,12.0,Finished Airing,"Oct 5, 2015",...,"Parody, Super Power",Seinen,24.0,R - 17+ (violence & profanity),8.511,19066.0,1112.0,4,2879907,59651
4,11757,Sword Art Online,"S.A.O, SAO",ソードアート・オンライン,Sword Art Online,Ever since the release of the innovative Nerve...,TV,25.0,Finished Airing,"Jul 8, 2012",...,"Love Polygon, Video Game",Unknown,23.0,PG-13 - Teens 13 or older,7.201,990254.0,29562.0,5,2813565,64997


In [5]:
df.columns

Index(['ID', 'Title', 'Synonyms', 'Japanese', 'English', 'Synopsis', 'Type',
       'Episodes', 'Status', 'Start_Aired', 'End_Aired', 'Premiered',
       'Broadcast', 'Producers', 'Licensors', 'Studios', 'Source', 'Genres',
       'Themes', 'Demographics', 'Duration_Minutes', 'Rating', 'Score',
       'Scored_Users', 'Ranked', 'Popularity', 'Members', 'Favorites'],
      dtype='object')

**Columns explaination:**
- Broadcast: The day and time of the week that the anime is broadcast
- Score: The average score on MyAnimeList
- Score_Users: Number of user that scored the anime
- Members: The number of members of MyAnimeList who have added this anime to their list
- Favourite: The number of members of this website who have added this anime to their favourites.

# **Data Cleaning**

In [6]:
# Only keep columns that are relevant to the analysis
df = df[['Title', 'English', 'Type', 'Episodes', 'Start_Aired', 'End_Aired', 'Status', 'Producers', 'Studios', 'Source', 'Genres',
       'Themes', 'Demographics', 'Duration_Minutes', 'Rating', 'Score',
       'Scored_Users', 'Ranked', 'Popularity', 'Members', 'Favorites']]

In [7]:
# Parse multi-value fields into arrays
# def parse_multi_value(field_value):
#     array_value = field_value.split(',')
#     if '' in array_value:
#         return []
#     return array_value

# cols = ['Producers', 'Studios', 'Genres', 'Themes', 'Demographics']
# for col in cols:
#     df[col] = df[col].apply(parse_multi_value)

### **Cleaning null values**

In [8]:
df.isnull().sum()

Title                  0
English                0
Type                   0
Episodes             547
Start_Aired            0
End_Aired              0
Status                 0
Producers              0
Studios                0
Source                 0
Genres                 0
Themes                 0
Demographics           0
Duration_Minutes     599
Rating               545
Score               6898
Scored_Users        6898
Ranked              1924
Popularity             0
Members                0
Favorites              0
dtype: int64

#### 1. Cleaning null Episodes values

In [9]:
# Checing if animes that has null Episodes value has the status "Not yet aired" or "Currently Airing" or not
df[df['Episodes'].isnull()]['Status'].value_counts()

Status
Not yet aired       328
Currently Airing    212
Finished Airing       7
Name: count, dtype: int64

In [10]:
# Check the animes that has null Episodes value and has the status "Finished Airing"
df[df['Episodes'].isnull() & (df['Status'] == 'Finished Airing')]

,Title,English,Type,Episodes,Start_Aired,End_Aired,Status,Producers,Studios,Source,...,Themes,Demographics,Duration_Minutes,Rating,Score,Scored_Users,Ranked,Popularity,Members,Favorites
16167,Momoya Norihei Anime CM,Unknown,Special,NaN,1958,2012,Finished Airing,Unknown,Unknown,Original,...,Parody,Unknown,30.0,G - All Ages,NaN,NaN,135772.0,16192,285,0
16802,Koishite!! Namashi-chan,Unknown,TV,NaN,"Jan 3, 2015",Unknown,Finished Airing,Unknown,Kachidoki Studio,Original,...,Unknown,Kids,1.0,G - All Ages,NaN,NaN,192892.0,16827,241,0
16863,Burutabu-chan,Unknown,TV,NaN,Unknown,Not Available,Finished Airing,Fuji TV,Unknown,Original,...,Unknown,Unknown,10.0,PG-13 - Teens 13 or older,NaN,NaN,172082.0,16887,237,0
17473,Konna Ko Iru kana,Unknown,TV,NaN,Apr 1986,Mar 1991,Finished Airing,NHK,Unknown,Unknown,...,Unknown,Kids,1.0,PG - Children,NaN,NaN,193282.0,17497,203,0
19438,Bobble Bobble Cook,Bobble Bobble Cook,TV,NaN,Mar 2010,Oct 2012,Finished Airing,Unknown,Unknown,Original,...,Unknown,Kids,23.0,PG - Children,NaN,NaN,171182.0,19465,76,0
19781,Jouiseon-in,Warriors in Black,TV,NaN,2008,Unknown,Finished Airing,Unknown,Unknown,Original,...,Unknown,Kids,13.0,G - All Ages,NaN,NaN,188472.0,19809,57,0
20179,Super Talk Talk,Super Talk Talk,TV,NaN,Unknown,Not Available,Finished Airing,Unknown,Unknown,Original,...,Unknown,Kids,14.0,PG - Children,NaN,NaN,153282.0,20207,48,0


These values are really weird, as I can't find the number episodes of these animes, despite their status is "Finished Airing". Hence, I've come up with the decision of deleting these values, as there's only seven of these anyway.

In [11]:
# Drop rows where 'Episodes' is null and 'Status' is 'Finished Airing'
df = df[~(df['Episodes'].isnull() & (df['Status'] == 'Finished Airing'))]

Regarding other null Episodes value, I will fill those values with the median episodes of their respective type (TV, Movie, etc).

In [12]:
df['Episodes'].fillna(df.groupby('Type')['Episodes'].transform('median'), inplace = True)

#### 2. Cleaning null Duration null values

In [13]:
# Do the same thing for 'Duration_Minutes' column
df['Duration_Minutes'].fillna(df.groupby('Type')['Duration_Minutes'].transform('median'), inplace = True)

#### 3. Cleaning Rating null values

In [14]:
df['Rating'].fillna('Unknown', inplace = True)

#### 4. Cleaning Score and Scored_Users null values

We can see that the number of null values of these two columns are the same, indicating that when Score is null, Scored_Users is also null, and vice versa. These rows likely represent anime that have not received any ratings. 

Given this, an approach could be considered is to fill the null values of Score with median valuee, and 0 for Scored_Users. Also, let's create a column indicating that an anime was scored or not.

In [15]:
df['Missing_Score'] = df['Score'].isnull().map({True: 1, False: 0})
df['Score'].fillna(df['Score'].median(), inplace = True)
df['Scored_Users'].fillna(0, inplace = True)

#### 5. Cleaning Ranked null values

In [16]:
# For the ranking, we will just put a very high number to indicate that the anime is not ranked
df['Ranked'].fillna(200000, inplace = True)

In [17]:
df.isnull().sum()

Title               0
English             0
Type                0
Episodes            0
Start_Aired         0
End_Aired           0
Status              0
Producers           0
Studios             0
Source              0
Genres              0
Themes              0
Demographics        0
Duration_Minutes    0
Rating              0
Score               0
Scored_Users        0
Ranked              0
Popularity          0
Members             0
Favorites           0
Missing_Score       0
dtype: int64

### **Handling Start_Aired and End_Aired data**
After some thorough inspection of the dataset, the date formatting is inconsistent throughout the dataset. 
- DD/MM/YYYY
- Month DD, YYYY
- YYYY

To use these dates effectively I need to standardize them into a single format. Let's try to take the year only.


In [18]:
def extract_year(date_str):
    if pd.isna(date_str):
        return None
    
    match = re.search(r'\d{4}', str(date_str))
    if match:
        return int(match.group(0))
    else:
        return None  

df['Aired_Year'] = df['Start_Aired'].apply(extract_year)
df['Aired_Year_Missing'] = df['Aired_Year'].isnull().astype(int)

df['End_Year'] = df['End_Aired'].apply(extract_year)
df['End_Year_Missing'] = df['End_Aired'].isnull().astype(int)

#Verify the results.
print(df['Aired_Year'].unique())
print(df['Aired_Year_Missing'].value_counts())

print(df['End_Year'].unique())
print(df['End_Year_Missing'].value_counts())

[2013. 2006. 2009. 2015. 2012. 2016. 2002. 2014. 2019. 2011. 2017. 2007.
 2018. 2008. 2020. 1999. 2010. 2004. 1998. 2001. 1995. 2003. 1997. 1989.
 2021. 2022. 1988. 1986. 2005. 2000. 1992. 1996. 1984. 1993.   nan 2023.
 1994. 1991. 1985. 1990. 1979. 1982. 1971. 1983. 1970. 1981. 1987. 1978.
 1980. 1973. 1974. 1977. 1972. 1976. 1969. 1967. 1963. 1975. 1917. 1961.
 1968. 1965. 1958. 1962. 1964. 1966. 1931. 1929. 1960. 1918. 1945. 1930.
 1943. 1956. 1957. 1959. 1947. 1928. 1926. 1934. 1932. 1935. 1933. 1924.
 1952. 1948. 1938. 1936. 1927. 1925. 1946. 1942. 1940. 1941. 1950. 1944.
 1939. 1937. 1949. 1954. 1953. 1955. 1951.]
Aired_Year_Missing
0    20982
1      471
Name: count, dtype: int64
[2013. 2007. 2010. 2015. 2012. 2016. 2014. 2019.   nan 2017. 2011. 2018.
 2009. 2021. 1999. 2008. 1996. 2020. 2004. 2005. 2006. 2022. 1989. 2001.
 2000. 1998. 2002. 1995. 1997. 2003. 1993. 1992. 1994. 1987. 1980. 1983.
 1972. 1986. 1971. 1988. 1991. 1978. 1981. 1990. 1979. 1974. 1973. 1985.
 1975. 1984. 

In [19]:
# The result was great. Hence, we could now drop the 'Start_Aired' column
df.drop('Start_Aired', axis=1, inplace=True)
df.drop('End_Aired', axis=1, inplace=True)

In [20]:
# Handle Aired and End Year
df['Aired_Year'].fillna(df['Aired_Year'].median(), inplace=True)
df['End_Year'].fillna(df['End_Year'].median(), inplace=True)

In [21]:
df.isnull().sum()

Title                 0
English               0
Type                  0
Episodes              0
Status                0
Producers             0
Studios               0
Source                0
Genres                0
Themes                0
Demographics          0
Duration_Minutes      0
Rating                0
Score                 0
Scored_Users          0
Ranked                0
Popularity            0
Members               0
Favorites             0
Missing_Score         0
Aired_Year            0
Aired_Year_Missing    0
End_Year              0
End_Year_Missing      0
dtype: int64

The dataset is looking great now I will export this to a csv and continue with visualizations in Power BI

In [22]:
df.to_csv('Anime_cleaned.csv', index=False)

In [23]:
df_copy = df.copy()